# import

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import KFold
import lightgbm as lgb
import random
from gensim.models.word2vec import Word2Vec 
import jieba
from tqdm.notebook import tqdm as tqdm
import time
from datetime import datetime
import IPython.display as ipd
from collections import *
import torch
from transformers import *
import torch.nn as nn
import torch.utils.data as Data
import math
import torch.nn.functional as F
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
from sklearn.utils.extmath import softmax

/home/cch/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
/home/cch/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/cch/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/cch/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# Read

In [2]:
VAR_PATH = '/data/cch/hyr/weibo/var'
df_weibo = pickle.load(open('%s/df_all.pickle'%VAR_PATH, 'rb'))

In [3]:
DATA_PATH = '/data/cch/weibo'
df_user_profile = pd.read_csv('%s/user_profile.csv'%DATA_PATH, sep='\t')

In [4]:
df_24h_feature_predict = pickle.load(open('%s/df_24h_feature_predict.pickle'%VAR_PATH, 'rb'))

In [5]:
# user_set = set(df_weibo['UserId'])
# df_user_profile[df_user_profile['UserId'].apply(lambda x : x in user_set)]

# 数据预处理

In [6]:
feature_24h_list = list(df_24h_feature_predict.columns)

continue_feature_list = [

    #文本硬特征
    'weibotext_len', 
    
    #性别
    'Gender', 
    
    #用户粉丝，关注数
    'follow_mean', 'follow_median', 'follow_max', 'follow_min', 'follower_mean',
    'follower_median', 'follower_max', 'follower_min',
    
    #user id target encode
#     'target_encode_0', 'target_encode_1', 'target_encode_2', 'target_encode_3', 'target_encode_4',
#     'target_encode_cnt', 
    
    #转发特征(微博维度)
     'repost_weibo_cnt_15_30_mins', 'repost_weibo_cnt_15_mins','repost_weibo_cnt_30_45_mins','repost_weibo_cnt_30_mins',
     'repost_weibo_cnt_45_60_mins','repost_weibo_cnt_45_mins','repost_weibo_cnt_60_mins',
     'repost_weibo_follow_max_max',
     'repost_weibo_follow_max_mean','repost_weibo_follow_max_min','repost_weibo_follow_max_sum','repost_weibo_follow_mean_max',
     'repost_weibo_follow_mean_mean','repost_weibo_follow_mean_min','repost_weibo_follow_mean_sum','repost_weibo_follow_median_max',
     'repost_weibo_follow_median_mean','repost_weibo_follow_median_min','repost_weibo_follow_median_sum',
     'repost_weibo_follow_min_max', 'repost_weibo_follow_min_mean','repost_weibo_follow_min_min','repost_weibo_follow_min_sum',
     'repost_weibo_follower_max_max','repost_weibo_follower_max_mean','repost_weibo_follower_max_min',
     'repost_weibo_follower_max_sum','repost_weibo_follower_mean_max','repost_weibo_follower_mean_mean',
     'repost_weibo_follower_mean_min','repost_weibo_follower_mean_sum','repost_weibo_follower_median_max',
     'repost_weibo_follower_median_mean','repost_weibo_follower_median_min','repost_weibo_follower_median_sum',
     'repost_weibo_follower_min_max','repost_weibo_follower_min_mean','repost_weibo_follower_min_min',
     'repost_weibo_follower_min_sum','repost_weibo_pass_time_max','repost_weibo_pass_time_mean','repost_weibo_pass_time_median',
     'repost_weibo_pass_time_min', 'repost_weibo_Verified', 'repost_weibo_Gender',
    
    #水军
     'repost_weibo_unique_repost_userid',
     'repost_weibo_repeat_repost_cnt',
     'repost_weibo_max_user_repost',
     'repost_weibo_other_cnt_max',
     'repost_weibo_other_cnt_min',
     'repost_weibo_other_cnt_mean',
     'repost_weibo_other_cnt_std',
     'repost_weibo_other_cnt_0',
     'repost_weibo_text_len_max',
     'repost_weibo_text_len_min',
     'repost_weibo_text_len_mean',
     'repost_weibo_text_len_std',

    #转发特征(用户维度)    
     'repost_userid_cnt_15_30_mins','repost_userid_cnt_15_mins','repost_userid_cnt_30_45_mins','repost_userid_cnt_30_mins',
     'repost_userid_cnt_45_60_mins','repost_userid_cnt_45_mins','repost_userid_cnt_60_mins','repost_userid_follow_max_max',
     'repost_userid_follow_max_mean','repost_userid_follow_max_min','repost_userid_follow_max_sum','repost_userid_follow_mean_max',
     'repost_userid_follow_mean_mean','repost_userid_follow_mean_min','repost_userid_follow_mean_sum',
     'repost_userid_follow_median_max','repost_userid_follow_median_mean','repost_userid_follow_median_min',
     'repost_userid_follow_median_sum','repost_userid_follow_min_max','repost_userid_follow_min_mean',
     'repost_userid_follow_min_min','repost_userid_follow_min_sum','repost_userid_follower_max_max',
     'repost_userid_follower_max_mean','repost_userid_follower_max_min','repost_userid_follower_max_sum',
     'repost_userid_follower_mean_max','repost_userid_follower_mean_mean','repost_userid_follower_mean_min',
     'repost_userid_follower_mean_sum','repost_userid_follower_median_max','repost_userid_follower_median_mean',
     'repost_userid_follower_median_min','repost_userid_follower_median_sum','repost_userid_follower_min_max',
     'repost_userid_follower_min_mean','repost_userid_follower_min_min','repost_userid_follower_min_sum',
     'repost_userid_pass_time_max','repost_userid_pass_time_mean','repost_userid_pass_time_median',
     'repost_userid_pass_time_min', 'repost_userid_Verified', 'repost_userid_Gender',
        
     'repost_userid_unique_repost_userid',
     'repost_userid_repeat_repost_cnt',
     'repost_userid_max_user_repost',
     'repost_userid_other_cnt_max',
     'repost_userid_other_cnt_mean',
     'repost_userid_other_cnt_std',
     'repost_userid_other_cnt_0',
     'repost_userid_text_len_max',
     'repost_userid_text_len_min',
     'repost_userid_text_len_mean',
     'repost_userid_text_len_std',
    
    
    #交叉特征 24h转发特征 target encode
#     'target_encode_24h_10_reposet_cnt',
#        'target_encode_24h_11_reposet_cnt', 'target_encode_24h_12_reposet_cnt',
#        'target_encode_24h_13_reposet_cnt', 'target_encode_24h_14_reposet_cnt',
#        'target_encode_24h_15_reposet_cnt', 'target_encode_24h_16_reposet_cnt',
#        'target_encode_24h_17_reposet_cnt', 'target_encode_24h_18_reposet_cnt',
#        'target_encode_24h_19_reposet_cnt', 'target_encode_24h_20_reposet_cnt',
#        'target_encode_24h_21_reposet_cnt', 'target_encode_24h_22_reposet_cnt',
#        'target_encode_24h_23_reposet_cnt', 'target_encode_24h_24_reposet_cnt',
#        'target_encode_24h_2_reposet_cnt', 'target_encode_24h_3_reposet_cnt',
#        'target_encode_24h_4_reposet_cnt', 'target_encode_24h_5_reposet_cnt',
#        'target_encode_24h_6_reposet_cnt', 'target_encode_24h_7_reposet_cnt',
#        'target_encode_24h_8_reposet_cnt', 'target_encode_24h_9_reposet_cnt'
]

cat_feature_list = [
    #
    '转发&点赞', '疫情', '特朗普|总统', '视频|链接', '粉|饭', '中国&金牌', '台湾',
    '历史', '发展|经济', '推荐', '东京&奥运',    

    #发表日期特征
    'post_day', 'post_weekday', 'post_month', 'post_hour',
    
    #交叉特征
    'userid_idx', 
    'cross_userid_idx_post_hour',
    'cross_userid_idx_post_weekday', 
    'cross_repost_1h_cnt_idx_post_hour', 'cross_repost_1h_cnt_idx_post_weekday',
]

hidden_feature_list = [
    'weibotext_wv_embed', 'weibotext_tfidf',#微博内容词向量/tfidf+svd降维
    'user_intro_wv_embed', 'user_intro_tfidf'#用户个性签名词向量/tfidf+svd降维
]


def generate_one_hot(df_weibo):
    global cat_feature_list
    
    cut_one_hot_feature_list = [    
        #转发特征(微博维度)
         'repost_weibo_cnt_15_30_mins', 'repost_weibo_cnt_15_mins','repost_weibo_cnt_30_45_mins','repost_weibo_cnt_30_mins',
         'repost_weibo_cnt_45_60_mins','repost_weibo_cnt_45_mins','repost_weibo_cnt_60_mins','repost_weibo_follow_max_max',
         'repost_weibo_follow_max_mean','repost_weibo_follow_max_min','repost_weibo_follow_max_sum','repost_weibo_follow_mean_max',
         'repost_weibo_follow_mean_mean','repost_weibo_follow_mean_min','repost_weibo_follow_mean_sum','repost_weibo_follow_median_max',
         'repost_weibo_follow_median_mean','repost_weibo_follow_median_min','repost_weibo_follow_median_sum',
         'repost_weibo_follow_min_max', 'repost_weibo_follow_min_mean','repost_weibo_follow_min_min','repost_weibo_follow_min_sum',
         'repost_weibo_follower_max_max','repost_weibo_follower_max_mean','repost_weibo_follower_max_min',
         'repost_weibo_follower_max_sum','repost_weibo_follower_mean_max','repost_weibo_follower_mean_mean',
         'repost_weibo_follower_mean_min','repost_weibo_follower_mean_sum','repost_weibo_follower_median_max',
         'repost_weibo_follower_median_mean','repost_weibo_follower_median_min','repost_weibo_follower_median_sum',
         'repost_weibo_follower_min_max','repost_weibo_follower_min_mean','repost_weibo_follower_min_min',
         'repost_weibo_follower_min_sum','repost_weibo_pass_time_max','repost_weibo_pass_time_mean','repost_weibo_pass_time_median',
         'repost_weibo_pass_time_min', 'repost_weibo_Verified', 'repost_weibo_Gender',

         'repost_weibo_unique_repost_userid',
         'repost_weibo_repeat_repost_cnt',
         'repost_weibo_max_user_repost',
         'repost_weibo_other_cnt_max',
         'repost_weibo_other_cnt_min',
         'repost_weibo_other_cnt_mean',
         'repost_weibo_other_cnt_std',
         'repost_weibo_other_cnt_0',
         'repost_weibo_text_len_max',
         'repost_weibo_text_len_min',
         'repost_weibo_text_len_mean',
         'repost_weibo_text_len_std',

        #转发特征(用户维度)    
         'repost_userid_cnt_15_30_mins','repost_userid_cnt_15_mins','repost_userid_cnt_30_45_mins','repost_userid_cnt_30_mins',
         'repost_userid_cnt_45_60_mins','repost_userid_cnt_45_mins','repost_userid_cnt_60_mins','repost_userid_follow_max_max',
         'repost_userid_follow_max_mean','repost_userid_follow_max_min','repost_userid_follow_max_sum','repost_userid_follow_mean_max',
         'repost_userid_follow_mean_mean','repost_userid_follow_mean_min','repost_userid_follow_mean_sum',
         'repost_userid_follow_median_max','repost_userid_follow_median_mean','repost_userid_follow_median_min',
         'repost_userid_follow_median_sum','repost_userid_follow_min_max','repost_userid_follow_min_mean',
         'repost_userid_follow_min_min','repost_userid_follow_min_sum','repost_userid_follower_max_max',
         'repost_userid_follower_max_mean','repost_userid_follower_max_min','repost_userid_follower_max_sum',
         'repost_userid_follower_mean_max','repost_userid_follower_mean_mean','repost_userid_follower_mean_min',
         'repost_userid_follower_mean_sum','repost_userid_follower_median_max','repost_userid_follower_median_mean',
         'repost_userid_follower_median_min','repost_userid_follower_median_sum','repost_userid_follower_min_max',
         'repost_userid_follower_min_mean','repost_userid_follower_min_min','repost_userid_follower_min_sum',
         'repost_userid_pass_time_max','repost_userid_pass_time_mean','repost_userid_pass_time_median',
         'repost_userid_pass_time_min', 'repost_userid_Verified', 'repost_userid_Gender',

         'repost_userid_unique_repost_userid',
         'repost_userid_repeat_repost_cnt',
         'repost_userid_max_user_repost',
         'repost_userid_other_cnt_max',
    #      'repost_userid_other_cnt_min',
         'repost_userid_other_cnt_mean',
         'repost_userid_other_cnt_std',
         'repost_userid_other_cnt_0',
         'repost_userid_text_len_max',
         'repost_userid_text_len_min',
         'repost_userid_text_len_mean',
         'repost_userid_text_len_std',
    ]

    one_hot_feature_name_list = []
    df_one_hot_list = []
    for f in cat_feature_list:
        df_one_hot = pd.get_dummies(df_weibo[f])
        one_hot_feature_name = ['%s_%s'%(f, name) for name in df_one_hot.columns]
        one_hot_feature_name_list.extend(one_hot_feature_name)
        df_one_hot.columns = one_hot_feature_name 
        df_one_hot_list.append(df_one_hot)

    cut_num = 5
    for f in cut_one_hot_feature_list:
        df_one_hot = pd.get_dummies(pd.qcut(df_weibo[f], cut_num, duplicates="drop"))
        one_hot_feature_name = ['%s_qcut_%s'%(f, name) for name in df_one_hot.columns]
        one_hot_feature_name_list.extend(one_hot_feature_name)
        df_one_hot.columns = one_hot_feature_name 
        df_one_hot_list.append(df_one_hot)

        df_one_hot = pd.get_dummies(pd.cut(df_weibo[f], cut_num))
        one_hot_feature_name = ['%s_cut_%s'%(f, name) for name in df_one_hot.columns]
        one_hot_feature_name_list.extend(one_hot_feature_name)
        df_one_hot.columns = one_hot_feature_name 
        df_one_hot_list.append(df_one_hot)
    return pd.concat(df_one_hot_list, 1)

In [7]:

df_weibo.index = df_weibo.WeiboId

# df_weibo['sample_weight'] = df_weibo['label'].map({0:1,1:10,2:50,3:100,4:300,-1:0})
df_weibo['sample_weight'] = df_weibo['label'].map({0:0.00333333,1:0.03333333,2:0.16666667,3:0.33333333,4:1,-1:0})
df_weibo['label_10'] = df_weibo['ForwordCount'].apply(lambda x : int(x//10) if x < 300 else 30).value_counts()

df_weibo_one_hot = generate_one_hot(df_weibo)
one_hot_feature_list = list(df_weibo_one_hot.columns)
df_weibo = pd.concat([df_weibo, df_weibo_one_hot], 1)
df_weibo = pd.concat([df_weibo, df_24h_feature_predict], 1)



dense_mean = np.array(df_weibo[continue_feature_list].mean()) 
dense_std = np.array(df_weibo[continue_feature_list].std())

forword_cnt_mean = df_weibo.query('type=="train"')['ForwordCount'].mean()
forword_cnt_std = df_weibo.query('type=="train"')['ForwordCount'].std()

mp_userid_idx = dict([(userid,i) for i, userid in enumerate(set(df_weibo['UserId']))])

# train_x = np.array(df_weibo.query('type=="train"')[continue_feature_list]).astype('float32')
# train_label = np.array(df_weibo.query('type=="train"')['label']).astype('int')
# train_forward_cnt = np.array(df_weibo.query('type=="train"')['ForwordCount']).astype('float32')

# test_x = np.array(df_weibo.query('type=="test"')[continue_feature_list]).astype('float32')

# train_x, test_x = (train_x - mean)/std, (test_x - mean)/std

# train_x.shape, train_y.shape, test_x.shape

/home/cch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:198: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
/home/cch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.
/home/cch/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


In [8]:
def get_distibution(cnt):
    ret = np.zeros(5)
    if cnt > 400:
        cnt = 400
    ll = [-10,11,51,151,301]
    rr = [10,50,150,300,500]
    median = [(l+r)//2 for l,r in zip(ll, rr)]
    interval_length = [r-l+3 for l, r in zip(ll, rr)]
    eps = 1e-8
    for i in range(5):
        dis = abs(cnt - median[i])/interval_length[i]
        ret[i] = 1/(dis+eps)
    return ret/sum(ret)

def get_distibution_softmax(cnt):
    ret = np.zeros(5)
    if cnt > 400:
        cnt = 400
    ll = [-10,11,51,151,301]
    rr = [10,50,150,300,500]
    median = [(l+r)//2 for l,r in zip(ll, rr)]
    interval_length = [r-l+3 for l, r in zip(ll, rr)]
    eps = 1e-8
    for i in range(5):
        dis = abs(cnt - median[i])/interval_length[i]
        ret[i] = dis
    return softmax([np.array(-ret)])[0]

def get_mp_distibution(func_name):
    ret = {}
    for i in range(500):
        ret[i] = func_name(i)
    return ret

mp_distibution = get_mp_distibution(get_distibution)
mp_distibution_softmax = get_mp_distibution(get_distibution_softmax)

# Dataset

In [9]:
class WeiboDataset(Data.Dataset):
    def __init__(self, weibo_id):
        self.weibo_id = weibo_id
        
    def __len__(self):
        return len(self.weibo_id)
    
    def __getitem__(self,idx):
        return self.weibo_id[idx]

def collate_fn(weibo_ids, cal_loss=True):

    global dense_mean, dense_std, df_weibo, albert_tokenizer, mp_userid_idx, one_hot_feature_list, mp_distibution
        
    df_weibo_sub = df_weibo.loc[weibo_ids]
    #连续性特征归一化
    dense = (np.array(df_weibo_sub[continue_feature_list].values) - dense_mean) /dense_std 
    soft_label = \
    np.array(list(df_weibo_sub['ForwordCount'].apply(lambda x : mp_distibution.get(x,-1) if x < 400 else mp_distibution[400])))
    
    #标签
    y = np.array(df_weibo_sub['label'])
    forword_cnt = np.array(df_weibo_sub['ForwordCount'])
    #样本权重
    sample_weight = np.array(df_weibo_sub['sample_weight'])
    
    #序列文本特征用于bert encoder
    #微博正文
#     df_weibo_text = df_weibo_sub['WeiboText']
#     max_length = min(args.weibo_text_max_length, max(df_weibo_text.apply(len)))
#     weibotext_tokenized_list = []
#     for sentence in df_weibo_text:
#         sentence = list(sentence)
#         if len(sentence) > max_length:
#             text = sentence[:max_length]
#         else:
#             text = sentence + ['<pad>']*(max_length-len(sentence))
#         weibotext_tokenized_list.append(albert_tokenizer.encode(text))        
    
    #用户简介
#     max_length = max(df_weibo_sub['intro'].apply(len))
#     intro_tokenized_list = []
#     for sentence in df_weibo_sub['intro']:
#         sentence = list(sentence)
#         if len(sentence) > max_length:
#             text = sentence[:max_length]
#         else:
#             text = sentence + ['<pad>']*(max_length-len(sentence))
#         intro_tokenized_list.append(albert_tokenizer.encode(text))        
    
    #固定文本特征
    text_feature_list = []
    for f in ['weibotext_wv_embed', 'weibotext_tfidf', 'user_intro_wv_embed', 'user_intro_tfidf']:
        hidden_train = np.array(list(df_weibo_sub[f]))
        text_feature_list.append(hidden_train)
    text_hidden = np.concatenate(text_feature_list, 1)
    
    user_id_token = np.array(df_weibo_sub['UserId'].map(mp_userid_idx))
    
    #转发用户+转发文本 聚合
    max_repost_user_set = 512
    np_repost_text_wv = np.zeros((len(weibo_ids), max_repost_user_set, 10))
    np_repost_user_dense = np.zeros((len(weibo_ids), max_repost_user_set, 8))
    np_repost_user_id = np.zeros((len(weibo_ids), max_repost_user_set)).astype('int')
    np_repost_set_len = np.zeros((len(weibo_ids),))
    for i, weiboid in enumerate(weibo_ids):
        userid_set_len = min(max_repost_user_set, df_weibo_sub.at[weiboid, 'repost_weibo_set_len'])
        np_repost_text_wv[i,:userid_set_len,:] = np.array(df_weibo_sub.at[weiboid, 'repost_weibo_repost_text_wv'])[:userid_set_len]
        np_repost_user_id[i,:userid_set_len] = np.array(df_weibo_sub.at[weiboid, 'repost_weibo_repost_user_id_freq'])[:userid_set_len]
        np_repost_set_len[i] = userid_set_len
        np_repost_user_dense[i,:userid_set_len] = np.array(df_weibo_sub.at[weiboid, 'repost_weibo_set_user_dense'])[:userid_set_len]
    
    return {
        'cal_loss':cal_loss,
        'x_dense':torch.tensor(dense).float().to(args.device), 
        'label':torch.tensor(y).long().to(args.device),
        'label_10':torch.tensor(np.array(df_weibo_sub['label_10'])).long().to(args.device),
        'forword_cnt' : torch.tensor(forword_cnt).float().to(args.device),
        'sample_weight':torch.tensor(sample_weight).float().to(args.device),
#         'weibo_text_token':torch.tensor(weibotext_tokenized_list).to(args.device),
#         'intro_text_token':torch.tensor(intro_tokenized_list).to(args.device),
        'text_wv_tfidf':torch.tensor(text_hidden).float().to(args.device),
        'user_id_token':torch.tensor(user_id_token).long().to(args.device),
        
        'repost_set_text_wv':torch.tensor(np_repost_text_wv).float().to(args.device),
        'repost_set_userid':torch.tensor(np_repost_user_id).long().to(args.device),
        'repost_set_len' : torch.tensor(np_repost_set_len).long().to(args.device),
        'repost_user_dense':torch.tensor(np_repost_user_dense).float().to(args.device),
        'one_hot_feature':torch.tensor(np.array(df_weibo_sub[one_hot_feature_list])).float().to(args.device),
        'soft_label':torch.tensor(soft_label).float().to(args.device),
    }

# train_dataset=WeiboDataset(train_weibo_id_list)
# data_loader = Data.DataLoader(
#     dataset=train_dataset,      
#     batch_size=args.batch_size,      
#     shuffle=True,
#     collate_fn=collate_fn,
#     num_workers = args.n_worker,
# )
# for _ in data_loader:
#     break



# Model

## model

In [14]:
class GeLU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1. + torch.tanh(x * 0.7978845608 * (1. + 0.044715 * x * x)))

class Linear(nn.Module):
    def __init__(self, in_features, out_features, activations=False):
        super().__init__()
        linear = nn.Linear(in_features, out_features)
        nn.init.normal_(linear.weight, std=math.sqrt((2. if activations else 1.) / in_features))
        nn.init.zeros_(linear.bias)
        modules = [nn.utils.weight_norm(linear)]
        if activations:
            modules.append(GeLU())
        self.model = nn.Sequential(*modules)

    def forward(self, x):
        return self.model(x)

class CeLossOut(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.out_label_10 = Linear(in_features, 31)
        self.out = Linear(in_features, 5)
    
    def forward(self, x, input_dict):
#         logit_label_10 = self.out_label_10(x)
        logit = self.out(x)
        loss = None
        if input_dict['cal_loss']:
#             loss_func = nn.CrossEntropyLoss().to(args.device)
#             loss_label10 = loss_func(logit_label_10, input_dict['label_10'])
            
            loss_func = nn.CrossEntropyLoss(reduction='none').to(args.device)
            loss_arr = loss_func(logit, input_dict['label'])
            loss = (loss_arr * input_dict['sample_weight']).mean()
        return loss, torch.nn.Softmax(1)(logit)

class PoissonLossOut(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.out = Linear(in_features, 1)
    
    def forward(self, x, input_dict):
        logit = self.out(x)
        loss = None
        if input_dict['cal_loss']:
            loss_func = nn.PoissonNLLLoss(log_input=True, reduction='none')
            max_v = 11
            logit[logit>max_v] = max_v
            loss_arr = loss_func(logit, input_dict['forword_cnt'].unsqueeze(-1)).squeeze(-1)
            loss = (loss_arr * input_dict['sample_weight']).mean()        
        return loss, torch.exp(logit)

class MSELossOut(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.out = Linear(in_features, 1)
    
    def forward(self, x, input_dict):
        global forword_cnt_mean, forword_cnt_std
        logit = self.out(x)
        loss = None
        if input_dict['cal_loss']:
            norm_target = (input_dict['forword_cnt']-forword_cnt_mean)/forword_cnt_std
#             norm_target = torch.log(input_dict['forword_cnt'])
            loss_arr = (logit.squeeze(-1)-norm_target)**2
#             loss = (loss_arr * input_dict['sample_weight']).mean()
            loss = loss_arr.mean()
        
        logit = logit*forword_cnt_std + forword_cnt_mean
#         logit = torch.exp(logit)
        logit[logit < 0] = 0
        return loss, logit
    
class TargetLossOut(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.out = Linear(in_features, 5)
    
    def forward(self, x, input_dict):
        global forword_cnt_mean, forword_cnt_std
        logit = self.out(x)
        loss = None
        if input_dict['cal_loss']:
            logit_softmax = torch.nn.Softmax(1)(logit) 
#             logit_softmax = torch.log(logit_softmax+0.001)
            score_arr = logit_softmax * torch.tensor([0.00333333,0.03333333,0.16666667,0.33333333,1]).to(logit_softmax.device)
            score_arr = score_arr[:, input_dict['label']] * torch.eye(input_dict['label'].shape[0]).to(logit_softmax.device)
            loss = -torch.sum(score_arr)/torch.sum(input_dict['sample_weight'])
            
        return loss, logit 
    
class SoftCeLossOut(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.out = Linear(in_features, 5)
    
    def forward(self, x, input_dict):
        logit = self.out(x)
        loss = None
        logit = torch.nn.Softmax(1)(logit) 

        if input_dict['cal_loss']:
            logit_softmax = torch.log(logit+0.001)
#             score_arr = logit_softmax * torch.tensor([0.00333333,0.03333333,0.16666667,0.33333333,1]).to(logit_softmax.device)
#             print(input_dict['soft_label'])
#             print(score_arr.shape, input_dict['soft_label'].shape)
            score_arr = logit_softmax * input_dict['soft_label']
            score_arr = score_arr.sum(1) * input_dict['sample_weight']
#             print(score_arr.shape)
            loss = -torch.mean(score_arr)
        return loss, logit

class MLPAttentionPool(nn.Module):
    def __init__(self,key_size,units):
        super().__init__()
        self.proj = nn.Sequential(nn.Linear(key_size,units,bias=False),
                                  nn.Tanh(),
                                  nn.Linear(units,1,bias=False))
        
    def masked_softmax_1d(self, X, valid_len):
        if valid_len is None:
            return F.softmax(X,dim=-1), _
        else:
            shape=X.shape
            if valid_len.dim()==1:
                valid_len=valid_len.view(-1,1).repeat(1,shape[1])
            
            mask=(torch.arange(0,X.shape[-1]).to(X.device).repeat(X.shape[0],1)<valid_len).bool()
            
            X = X.masked_fill_(~mask, -float('inf'))
            return F.softmax(X,dim=-1).view(shape), mask

    def forward(self, key, valid_len):
        scores = self.proj(key).squeeze(-1)
        attention_weights, mask = self.masked_softmax_1d(scores,valid_len)
        seq_out = attention_weights.unsqueeze(-1) * key
        return seq_out.sum(1)
    
class RepostSetEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        user_id_embed = 10
        text_embed = 10
        user_dense_embed = 8
        self.user_embeding = nn.Embedding(4000, user_id_embed)        
        self.attention_pool = MLPAttentionPool(user_id_embed + text_embed + user_dense_embed, 64)
        
    def forward(self, input_dict):
        x_set_wv = input_dict['repost_set_text_wv']
        x_set_user_embed = self.user_embeding(input_dict['repost_set_userid'])
        x_set_user_dense = input_dict['repost_user_dense']
        x = torch.cat([x_set_wv, x_set_user_embed, x_set_user_dense], -1)
        
        return self.attention_pool(x, input_dict['repost_set_len'])
    
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        global continue_feature_list, one_hot_feature_list
        
        in_features = len(continue_feature_list)
        bert_encode_feature = 312
        bert_encode_feature = 0
        text_wv_tfidf_feature = 40
        user_id_embed = 10
        repost_set_embed = 28
#         repost_set_embed = 0
        one_hot_feature = len(one_hot_feature_list)
        
#         global pretrained
#         self.albert_weibotext = BertModel.from_pretrained(pretrained).to(args.device)
#         self.albert_intro = BertModel.from_pretrained(pretrained).to(args.device)
        
#         self.transformer_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=bert_encode_feature, nhead=8), 
#                                                     num_layers=1)

        self.user_embeding = nn.Embedding(90, user_id_embed)
        
        self.repost_set_encoder = RepostSetEncoder().to(args.device)

        self.fc = Linear(in_features, args.embedding_size, True)
        
        self.other_fc = Linear(2*bert_encode_feature+text_wv_tfidf_feature + user_id_embed + repost_set_embed + one_hot_feature,
                         args.embedding_size, True)

        self.fc_out = Linear(2*args.embedding_size, args.embedding_size, True)
        
        self.dp1 = torch.nn.Dropout(args.dropout_rate)
        self.dp2 = torch.nn.Dropout(args.dropout_rate)

        self.out_layer  = {
                'ce': CeLossOut(args.embedding_size),
                'poisson': PoissonLossOut(args.embedding_size),
                'mse' : MSELossOut(args.embedding_size),
                'target' : TargetLossOut(args.embedding_size),
                'soft_ce':SoftCeLossOut(args.embedding_size),
        }[args.loss]
        
                
    def forward(self, input_dict):
#         x_weibotext = self.albert_weibotext(input_dict['weibo_text_token'])[1]
#         x_intro = self.albert_intro(input_dict['intro_text_token'])[1]
#         x_seq = self.transformer_encoder(x_seq)
#         x_text_out = torch.max(x_seq, 1)[0]
#         print(x_bert.shape)
        x_user_embed = self.user_embeding(input_dict['user_id_token'])
        
        x_repost_embeding = self.repost_set_encoder(input_dict)
        
        x = self.other_fc(torch.cat([input_dict['text_wv_tfidf'], x_user_embed,
                       x_repost_embeding, input_dict['one_hot_feature']], 1))
        x = torch.cat([x, self.fc(input_dict['x_dense'])], 1)
        x = self.dp1(x)
        x = self.fc_out(x)
#         x = self.dp2(x)
        return self.out_layer(x, input_dict)



In [15]:
'''
评估函数传入的是1，2，3，4，5五个档位的值，注意从1开始

'''

def precision_score_hyr(predictions, ground_truths):
    predictions, ground_truths = np.array(predictions)-1, np.array(ground_truths)-1
    
    score, total = 0, 0
    for p_cnt, g_cnt in zip(predictions, ground_truths):
        if g_cnt==0:
            total += 1
            if p_cnt == g_cnt:
                score += 1
        elif g_cnt==1:
            total += 10
            if p_cnt == g_cnt:
                score += 10
        elif  g_cnt==2:
            total += 50
            if p_cnt == g_cnt:
                score += 50
        elif  g_cnt==3:
            total += 100
            if p_cnt == g_cnt:
                score += 100
        elif  g_cnt==4:
            total += 300
            if p_cnt == g_cnt:
                score += 300
    return score/total

def precision_score_cch(predictions, ground_truths):
    
    predictions, ground_truths = np.array(predictions)-1, np.array(ground_truths)-1
    y_pred = predictions
    y_true = ground_truths

    w=[1,10,50,100,300]
    n = len(y_true)
    count_r = [0 for i in range(5)]
    count = [0 for i in range(5)]
    for i in range(n):
        count[y_true[i]] += 1
        if y_pred[i] == y_true[i]:
            count_r[y_pred[i]] += 1
    sum1 = sum(w[i]*count_r[i] for i in range(5))
    sum2 = sum(w[i]*count[i] for i in range(5))
    precision = sum1/sum2
    return precision

## 训练过程

In [16]:
def train_nn(weibo_id_list):
    train_dataset=WeiboDataset(weibo_id_list)
    data_loader = Data.DataLoader(
        dataset=train_dataset,      
        batch_size=args.batch_size,      
        shuffle=True,
        collate_fn=collate_fn,
        num_workers = args.n_worker,
    )
    
    model = DNN().to(args.device)
        
    no_decay = ["bias", "gamma","beta"]
    optimizer_grouped_parameters = [
        {
            "params": [p for name, p in model.named_parameters() if 'albert' not in name],
            "lr": args.lr,
        },
        {
            "params": [p for name, p in model.named_parameters() if 'albert' in name],
            "lr": args.fine_tune_layer_lr,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr = args.lr, weight_decay = args.weight_decay)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(len(weibo_id_list)//(args.batch_size)),
        num_training_steps=int(len(weibo_id_list) / args.batch_size * args.epoch)
    )
    
    
    for _ in range(args.epoch):
        for input_dict in tqdm(data_loader):
#             print(word_matrix)
            loss, logit = model(input_dict)            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5)
            optimizer.step()
            scheduler.step()
            if args.debug:
                break

    return model

def forward_nn(model, weibo_id_list):
    model.eval()
    train_dataset=WeiboDataset(weibo_id_list)
    data_loader = Data.DataLoader(
        dataset=train_dataset,      
        batch_size=args.batch_size,      
        shuffle=False,
        collate_fn=lambda x:collate_fn(x,False),
        num_workers = args.n_worker,
    )
    with torch.no_grad():
        pre_list = []
        for input_dict in tqdm(data_loader):
            _, logit = model(input_dict)
            pre_list.append(logit.cpu().detach().numpy())
    return np.concatenate(pre_list)

def cross_validation_nn(train_weibo_id_list, test_weibo_id_list):
    n_flod = args.n_flod
    folds = KFold(n_splits=n_flod, shuffle=True, random_state=SEED)
    train_weibo_id_list = np.array(train_weibo_id_list)
    test_weibo_id_list = np.array(test_weibo_id_list)
    
    
    score_train = np.zeros((len(train_weibo_id_list), LOSS.out_feature_dim))
    score_test = np.zeros((len(test_weibo_id_list), LOSS.out_feature_dim))
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_weibo_id_list, train_weibo_id_list)):
        
        trn_weibo_id_list = train_weibo_id_list[trn_idx]
        val_weibo_id_list = train_weibo_id_list[val_idx]
        model = train_nn(trn_weibo_id_list)
        
        model.eval()
        trn_logit = forward_nn(model, trn_weibo_id_list)
        val_logit = forward_nn(model, val_weibo_id_list)
        
#         print(trn_logit)
#         print(pd.Series([count2idx(num)+1 for num in trn_logit[:, 0]]).value_counts())
        
        ipd.display(pd.DataFrame({
            'train': [
                precision_score_cch(LOSS.logit2count(trn_logit), 
                                    np.array(df_weibo.loc[trn_weibo_id_list]['label']).astype('int')+1)
            ],
            'val': [
                precision_score_cch(LOSS.logit2count(val_logit), 
                                    np.array(df_weibo.loc[val_weibo_id_list]['label']).astype('int')+1),
                     
                ],
        }))
        
        
        score_train[val_idx] = val_logit

        score_test += forward_nn(model, test_weibo_id_list)/n_flod

    return score_train, score_test

# 主流程

In [22]:
# for loss_name in ['ce', 'poisson', 'mse', 'target']:
# for loss_name in ['ce', 'poisson', 'mse']:
for loss_name in ['ce']:
    ARG = namedtuple('ARG', [
        'batch_size',
        'epoch',
        'lr',
        'fine_tune_layer_lr',
        'weight_decay',
        'dropout_rate',
        'n_worker',
        'device',
        'embedding_size',
        'loss',
        'weibo_text_max_length',
        'n_flod',
        'debug',
    ])

    args = ARG(
        batch_size = 64,
        epoch = 5,
        lr = 0.006,
        fine_tune_layer_lr=2e-5,
        weight_decay = 0.01,
        dropout_rate = 0.1,
        n_worker = 0,
        device=torch.device("cuda:0"),
    #     device=torch.device("cpu"),
        embedding_size = 100,
        loss = loss_name, #ce poisson mse target,
        weibo_text_max_length = 500,
        n_flod = 5,
        debug = False
    )

    def setup_seed(seed):
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
    # 设置随机数种子
    SEED = 42
    setup_seed(SEED)

    def count2idx(num):
        if 0 <= num <= 10:
            return 0
        elif 10 <= num <= 50:
            return 1
        elif 50 <= num <= 150:
            return 2
        elif 150 <= num <= 300:
            return 3
        elif num >= 300:
            return 4
        if num < 0:
            print(num)
            return 0
        print(num)

    LOSS_PARAMETER = namedtuple('LOSS_PARAMETER',[
        'out_feature_dim',
        'logit2count',
    ])

    LOSS = LOSS_PARAMETER(
        out_feature_dim={
            'ce':5,
            'poisson':1,
            'mse':1,
            'target':5,
            'soft_ce':5
        }[args.loss],

        logit2count={
            'ce':lambda logit: np.argmax(logit,1)+1,
            'poisson':lambda logit:[count2idx(num)+1 for num in logit[:, 0]],
            'mse':lambda logit:[count2idx(num)+1 for num in logit[:, 0]],
            'target':lambda logit: np.argmax(logit,1)+1,
            'soft_ce':lambda logit: np.argmax(logit,1)+1,
        }[args.loss]
    )

    pretrained = '/data/cch/hyr/weibo/voidful/albert_chinese_tiny'
    albert_tokenizer = BertTokenizer.from_pretrained(pretrained)
    # albert_model = BertModel.from_pretrained(pretrained).to(args.device)
    start = time.time()
    train_weibo_id_list = list(df_weibo.query('type == "train"').index)
    test_weibo_id_list = list(df_weibo.query('type == "test"').index)

    score_train, score_test =\
    cross_validation_nn(train_weibo_id_list, test_weibo_id_list)

    time.time() - start

    MODEL_OUT_PATH = '/data/cch/hyr/weibo/sub_model_output'

    ret_dict = {'WeiboId':train_weibo_id_list+test_weibo_id_list}
    ret_dict.update(
        dict([['nn_hyr_loss_%s_%d'%(args.loss, i), np.concatenate([score_train[:, i], score_test[:, i]])] \
              for i in range(LOSS.out_feature_dim)])
    )
    df_model_output = pd.DataFrame(ret_dict)

#     pickle.dump(df_model_output, open('%s/df_nn_loss_%s.pickle'%(MODEL_OUT_PATH, args.loss), 'wb'))
#     df_model_output = pickle.load(open('%s/df_nn_loss_%s.pickle'%(MODEL_OUT_PATH, args.loss), 'rb'))

    logit_feature_name = sorted(list( set(df_model_output.columns) - set(['WeiboId'])))
    score = precision_score_cch(LOSS.logit2count(np.array(df_model_output.iloc[:18000][logit_feature_name])), 
                       np.array(df_weibo.loc[train_weibo_id_list]['label']).astype('int')+1)
    
    logging.info('here here here %s_%f'%(loss_name, score))
    


/home/cch/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/home/cch/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in Num

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

...

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

,train,val
0,0.804927,0.664068


  0%|          | 0/37 [00:00<?, ?it/s]

2021-12-11 16:38:40,726 - INFO - here here here ce_0.671308


In [23]:
pickle.dump(df_model_output, open('%s/df_nn_loss_%s.pickle'%(MODEL_OUT_PATH, args.loss), 'wb'))

In [ ]:
predictions = np.argmax(np.array(df_model_output.iloc[:18000][logit_feature_name]), 1)
lables = df_weibo.iloc[:18000]['label']
forward_cnt = df_weibo.iloc[:18000]['ForwordCount']

df_analysis = pd.DataFrame({
    'WeiboId':df_weibo.iloc[:18000].index,
    'predictions':predictions,
    'lables':list(lables),
    'forward_cnt':list(forward_cnt)
})


0.675909